In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('sample keywords with entities.csv')

# read 'list string' as list for the following columns
import ast

columns = ['name', 'entity_type', 'salience', 'entity_sentiment_score', 'entity_sentiment_magnitude']

for column in columns:
    df[column] = df[column].apply(lambda x: ast.literal_eval(x) if type(x)==str else '')

df.sample(3)

,query,title,name,entity_type,salience,entity_sentiment_score,entity_sentiment_magnitude
20457,what is employee retention,What is employee retention? definition and mea...,"[employee retention, definition, meaning]","[OTHER, OTHER, OTHER]","[0.5862148404121399, 0.23478971421718597, 0.17...","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
16676,sensodyne complete protection review,Sensodyne Sensitivity Toothpaste for Sensitive...,"[Sensodyne, Sensitivity Toothpaste, Teeth]","[OTHER, OTHER, OTHER]","[0.6203917860984802, 0.284095823764801, 0.0955...","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
2091,auto owners insurance agent appointment,Auto-Owners Insurance: Home,"[Home, Auto-Owners Insurance]","[OTHER, ORGANIZATION]","[0.5346556305885315, 0.4653443396091461]","[0.0, 0.0]","[0.0, 0.0]"


In [2]:
# drop <people also ask> and <video pack> records
df = df.loc[~df['title'].isin(['People Also Ask', 'Video Pack'])]

# drop title = nan
df['type'] = df['title'].apply(lambda x: type(x)==str)
df = df[df['type'] == True]
df = df.drop(['type'], axis=1).reset_index(drop=True)

## Aggregate the variables at query level by migrating into a new dataframe

In [3]:
df_new = pd.DataFrame(df['query'].drop_duplicates()).reset_index(drop=True)

import itertools

items = ['name', 'entity_type', 'salience']
    
for item in items:
    
    # create new column
    df_new[item] = None
    
    # combine the lists for each query
    for i in range(df_new.shape[0]):
        
        query = df_new['query'][i]
        query_pos = df.loc[df['query'] == query, item]
    
        query_pos = sum(list(itertools.chain(query_pos)), [])
        df_new.loc[i, item] = query_pos

### Make a dictionary storing entities and their types for reference

In [4]:
title_entity_dictionary = pd.DataFrame()

items = ['name', 'entity_type']

for item in items:
    title_entity_dictionary[item] = sum(list(itertools.chain(df_new[item])), [])

title_entity_dictionary = title_entity_dictionary.drop_duplicates().reset_index(drop=True)
# title_entity_dictionary.to_csv('title entity dictionary.csv', index=False)
title_entity_dictionary.sample(3)

,name,entity_type
1927,Biotène,OTHER
2190,A Quote For Car Insurance | Right Now,OTHER
4303,Workplace Demands,OTHER


In [5]:
title_entity_dictionary['entity_type'].value_counts()

OTHER            4281
ORGANIZATION     1024
PERSON            543
CONSUMER_GOOD     456
WORK_OF_ART       438
LOCATION          352
NUMBER            300
EVENT              97
PRICE              68
DATE               33
ADDRESS            12
PHONE_NUMBER        5
Name: entity_type, dtype: int64

## Create Entity Type Features

In [6]:
df_new['price_phone'] = df_new['entity_type'].apply(lambda x: round(100*(x.count('PRICE')+x.count('PHONE_NUMBER'))/len(x), 2))

df_new['organization'] = df_new['entity_type'].apply(lambda x: round(100*x.count('ORGANIZATION')/len(x), 2))

df_new['work of art'] = df_new['entity_type'].apply(lambda x: round(100*x.count('WORK_OF_ART')/len(x), 2))

In [7]:
df_new['salience std'] = df_new['salience'].apply(lambda x: round(np.std(np.array(x), axis=0), 2))

In [8]:
df_new.sample(3)

,query,name,entity_type,salience,price_phone,organization,work of art,salience std
593,crest 3d white strips coupon printable,"[Crest Toothpaste, Coupons, Mouthwash, Whitest...","[OTHER, OTHER, CONSUMER_GOOD, LOCATION, OTHER,...","[0.5708202123641968, 0.2016894668340683, 0.172...",2.63,5.26,0.0,0.26
1700,what is the cause for dry mouth,"[Dry mouth, Symptoms, causes, Mayo Clinic, Dry...","[OTHER, OTHER, OTHER, ORGANIZATION, OTHER, OTH...","[0.5734050273895264, 0.18981856107711792, 0.17...",0.00,14.71,0.0,0.20
203,best auto home insurance bundle,"[ValuePenguin, Best Home, ValuePenguin, Best H...","[OTHER, OTHER, OTHER, OTHER, ORGANIZATION, OTH...","[0.695317268371582, 0.30468273162841797, 0.695...",0.00,15.15,0.0,0.31


In [9]:
df_new = df_new.drop(['name', 'entity_type', 'salience'], axis=1)

In [10]:
df_new.to_csv('entity for model.csv', index=False)